In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

###############################
# (1) Sentence Transformer 모델 로드
###############################
# 도메인 매칭에 강한 모델로 교체 (영어권 문서 검색 최적화 모델)
model_bert = SentenceTransformer('sentence-transformers/msmarco-distilbert-base-v4')



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/319 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
items = [
    # ==== (기존 6개 대응, item_id 67~72) =======================================
    {"item_id": 67, "title": "산악 체험 클럽", "desc": "높은 곳을 향해 도전하며 함께 즐기는 산행 활동", "personality": "외향형, 감정형"},
    {"item_id": 68, "title": "풋살 커뮤니티", "desc": "작은 필드에서 팀워크를 발휘하며 뛰는 풋살 모임", "personality": "외향형, 감정형"},
    {"item_id": 69, "title": "책사랑 스터디회", "desc": "다양한 장르의 도서를 읽고 심층 토론하는 모임", "personality": "내향형, 이성형"},
    {"item_id": 70, "title": "핸드메이드 공방", "desc": "직접 손으로 만드는 창작 활동을 배우는 스튜디오", "personality": "내향형, 이성형"},
    {"item_id": 71, "title": "e스포츠 챔피언십", "desc": "온라인 게임으로 대회와 이벤트를 진행하는 그룹", "personality": "외향형, 이성형"},
    {"item_id": 72, "title": "시네마 컬쳐클럽", "desc": "다양한 영화를 감상하고 문화를 공유하는 모임", "personality": "외향형, 감정형"},

    # ==== (운동 - 12개 대응, item_id 73~84) ====================================
    {"item_id": 73,  "title": "크로스핏 열정단",   "desc": "고강도 크로스핏 운동으로 체력을 단련하는 열정적 모임 (헬스장)", "personality": "외향형, 감정형"},
    {"item_id": 74,  "title": "명상 요가 서클",   "desc": "몸의 이완과 마음의 평화를 함께 추구하는 명상 요가 모임 (요가)", "personality": "외향형, 감정형"},
    {"item_id": 75,  "title": "코어 트레이닝 팸", "desc": "필라테스로 코어 근력과 유연성을 키우는 그룹 (필라테스)", "personality": "외향형, 감정형"},
    {"item_id": 76,  "title": "파도타기 워터팀",   "desc": "수영과 워터 스포츠를 함께 즐기며 체력을 키우는 모임 (수영)", "personality": "외향형, 감정형"},
    {"item_id": 77,  "title": "라켓 드림클럽",    "desc": "테니스를 비롯한 라켓 스포츠를 같이 배우고 즐기는 동호회 (테니스)", "personality": "외향형, 감정형"},
    {"item_id": 78,  "title": "필드 골프 리더스",   "desc": "골프 스윙 기술과 필드 매너를 함께 익히는 라운딩 팀 (골프)", "personality": "외향형, 감정형"},
    {"item_id": 79,  "title": "슬링 텀블러즈",     "desc": "암벽등반과 슬링 트레이닝으로 색다른 운동을 체험하는 그룹 (클라이밍)", "personality": "외향형, 감정형"},
    {"item_id": 80,  "title": "스포츠 골든리그",   "desc": "주중·주말에 축구 경기를 주기적으로 여는 커뮤니티 (축구)", "personality": "외향형, 감정형"},
    {"item_id": 81,  "title": "하프코트 슈터스",    "desc": "농구 코트에서 함께 뛰고 실력을 배양하는 팀 (농구)", "personality": "외향형, 감정형"},
    {"item_id": 82,  "title": "스트라이크 러버스",  "desc": "볼링 점수를 올리기 위해 함께 연구하고 즐기는 동아리 (볼링)", "personality": "외향형, 감정형"},
    {"item_id": 83,  "title": "스매싱 배드민턴",    "desc": "배드민턴 스매시 기술과 경기를 함께 연습하는 학습 모임 (배드민턴)", "personality": "외향형, 감정형"},
    {"item_id": 84,  "title": "나이트 러너스",      "desc": "야간 러닝을 통해 도시를 누비며 체력을 기르는 크루 (러닝)", "personality": "외향형, 감정형"},

    # ==== (여행 - 8개 대응, item_id 85~92) =====================================
    {"item_id": 85, "title": "오지여행 탐험대",      "desc": "국내 숨겨진 오지를 찾아 떠나며 모험심을 기르는 모임 (국내여행)", "personality": "외향형, 감정형"},
    {"item_id": 86, "title": "글로벌 백팩 트립",     "desc": "해외 배낭여행 루트를 함께 계획하고 실행하는 동호회 (해외여행)", "personality": "외향형, 감정형"},
    {"item_id": 87, "title": "필드 캠핑 어드벤처",    "desc": "백패킹과 캠핑을 결합해 자연에서 살아보는 체험 모임 (백패킹)", "personality": "외향형, 감정형"},
    {"item_id": 88, "title": "패밀리 캠핑 잼",       "desc": "가족 단위로 캠핑하며 자연 속 소통을 즐기는 그룹 (캠핑)", "personality": "외향형, 감정형"},
    {"item_id": 89, "title": "골목길 투어 클럽",      "desc": "도시 골목길과 역사적 공간을 함께 탐방하는 여행 서클 (도시여행)", "personality": "외향형, 감정형"},
    {"item_id": 90, "title": "미식헌터 크루",         "desc": "전국 맛집을 조사하고 직접 시식 투어를 떠나는 모임 (맛집탐방)", "personality": "외향형, 감정형"},
    {"item_id": 91, "title": "문화유산 탐방기획사",   "desc": "전통 문화유산과 축제를 기획·탐방하는 여행 동아리 (문화탐방)", "personality": "외향형, 감정형"},
    {"item_id": 92, "title": "숲속 힐링 트래블",      "desc": "자연 속에서 치유와 재충전을 목적으로 여행하는 그룹 (힐링여행)", "personality": "외향형, 감정형"},

    # ==== (독서 - 10개 대응, item_id 93~102) ==================================
    {"item_id": 93,  "title": "문학 리뷰 서클",       "desc": "소설과 문학작품을 읽고 비평하는 책동아리 (소설)", "personality": "내향형, 감정형"},
    {"item_id": 94,  "title": "시인 꿈나무 모임",     "desc": "서로의 시를 낭독하고 공유하는 창작 시클럽 (시)", "personality": "내향형, 감정형"},
    {"item_id": 95,  "title": "마음글쓰기 에세이회",   "desc": "에세이로 일상과 감정을 표현하며 소통하는 글모임 (에세이)", "personality": "내향형, 이성형"},
    {"item_id": 96,  "title": "실천형 자기계발단",     "desc": "자기계발서를 읽고 즉시 실천 계획을 세우는 스터디 (자기계발)", "personality": "내향형, 이성형"},
    {"item_id": 97,  "title": "깊이읽기 인문소사이어티", "desc": "인문학 텍스트를 심층 토론하며 사고력을 확장하는 그룹 (인문)", "personality": "내향형, 이성형"},
    {"item_id": 98,  "title": "역사공부 디스커션",     "desc": "시대별 역사적 사건을 공부하고 자유롭게 논의하는 모임 (역사)", "personality": "내향형, 이성형"},
    {"item_id": 99,  "title": "과학탐구 북클럽",       "desc": "최신 과학 서적을 중심으로 아이디어를 나누는 독서회 (과학)", "personality": "내향형, 이성형"},
    {"item_id": 100, "title": "경영전략 독해팀",       "desc": "경제/경영 관련 서적을 함께 분석하고 사례를 연구하는 그룹 (경제/경영)", "personality": "내향형, 이성형"},
    {"item_id": 101, "title": "철학강독 세미나",       "desc": "철학 원전을 읽고 핵심 개념을 토론하는 학술 모임 (철학)", "personality": "내향형, 이성형"},
    {"item_id": 102, "title": "아트북 인사이트회",      "desc": "예술 관련 서적을 통해 예술사와 작품 세계를 탐구하는 모임 (예술)", "personality": "내향형, 감정형"},

    # ==== (영화 - 10개 대응, item_id 103~112) =================================
    {"item_id": 103, "title": "낭만 영화 애호가들",     "desc": "감성적인 로맨스 영화를 좋아하는 사람들의 모임 (로맨스)", "personality": "외향형, 감정형"},
    {"item_id": 104, "title": "폭소 코미디 페스티벌",    "desc": "웃음을 주제로 코미디 영화를 함께 보는 소모임 (코미디)", "personality": "외향형, 감정형, 이성형"},
    {"item_id": 105, "title": "액션매니아즈",           "desc": "박진감 넘치는 액션 영화의 매력을 해부하는 그룹 (액션)", "personality": "외향형, 감정형"},
    {"item_id": 106, "title": "서스펜스 시네마분석단",    "desc": "스릴러 영화 속 긴장감과 스토리를 분석하는 모임 (스릴러)", "personality": "외향형, 감정형"},
    {"item_id": 107, "title": "호러매틱 스터디",        "desc": "공포영화의 분위기와 연출을 집중 탐구하는 팀 (공포)", "personality": "외향형, 감정형"},
    {"item_id": 108, "title": "SF 월드 디스커버리",     "desc": "공상과학 영화를 보고 미래 기술과 세계관을 토론하는 모임 (SF)", "personality": "외향형, 감정형"},
    {"item_id": 109, "title": "판타지 이매지네이션",     "desc": "환상적인 판타지 영화를 감상하며 아이디어를 공유하는 서클 (판타지)", "personality": "외향형, 감정형"},
    {"item_id": 110, "title": "휴먼드라마 인사이트",     "desc": "감동적인 드라마 장르 영화를 함께 보고 교훈을 찾는 모임 (드라마)", "personality": "외향형, 감정형"},
    {"item_id": 111, "title": "애니 애호회",            "desc": "다양한 애니메이션 영화를 정기 상영하며 감상하는 팬클럽 (애니메이션)", "personality": "외향형, 감정형"},
    {"item_id": 112, "title": "다큐 리뷰 포럼",         "desc": "다큐멘터리 영화를 보고 사회·환경 문제를 논의하는 모임 (다큐멘터리)", "personality": "외향형, 이성형"},

    # ==== (게임 - 10개 대응, item_id 113~122) =================================
    {"item_id": 113, "title": "RPG 모험단",         "desc": "판타지 세계에서 함께 파티를 이뤄 모험하는 RPG 팀 (RPG)", "personality": "외향형, 이성형"},
    {"item_id": 114, "title": "FPS 고인물 클랜",     "desc": "고난도 슈팅 기술을 연습하며 대회에도 나가는 FPS 동호회 (FPS)", "personality": "외향형, 이성형"},
    {"item_id": 115, "title": "액션코어 전략팀",      "desc": "액션 게임의 숨겨진 테크닉과 콤보를 연구하는 그룹 (액션)", "personality": "외향형, 이성형"},
    {"item_id": 116, "title": "전략모드 시뮬클럽",     "desc": "전략 시뮬레이션 장르를 심도 있게 플레이·분석하는 동호회 (전략)", "personality": "외향형, 이성형"},
    {"item_id": 117, "title": "시뮬레이션 마니아들",   "desc": "도시건설, 경영 시뮬레이션 등 다양한 시뮬 게임을 즐기는 모임 (시뮬레이션)", "personality": "외향형, 이성형, 감정형"},
    {"item_id": 118, "title": "스포츠게이머 챌린지",   "desc": "축구·농구 등 스포츠게임 리그를 운영하는 커뮤니티 (스포츠)", "personality": "외향형, 이성형"},
    {"item_id": 119, "title": "퍼즐 브레인홀릭",      "desc": "퍼즐·두뇌게임을 함께 풀며 아이디어를 공유하는 팀 (퍼즐)", "personality": "외향형, 이성형"},
    {"item_id": 120, "title": "리듬게임 오케스트라",    "desc": "음악/리듬게임에서 합주하는 느낌으로 협력 플레이하는 모임 (음악/리듬)", "personality": "외향형, 이성형"},
    {"item_id": 121, "title": "카드플레이 매니아",     "desc": "포커·TCG 등 카드 기반 게임을 심도 있게 즐기는 동아리 (카드)", "personality": "외향형, 이성형"},
    {"item_id": 122, "title": "MMORPG 연합길드",      "desc": "대규모 온라인 RPG에서 길드원들과 함께 도전하는 팀 (MMORPG)", "personality": "외향형, 이성형"},

    # ==== (공예 - 10개 대응, item_id 123~132) =================================
    {"item_id": 123, "title": "니트디자인 스터디오",    "desc": "뜨개질로 패션 소품과 의류를 창작하는 워크숍 (뜨개질)", "personality": "내향형, 이성형"},
    {"item_id": 124, "title": "자수 디테일연구회",      "desc": "섬세한 자수 기법을 연구하며 작품을 완성하는 모임 (자수)", "personality": "내향형, 이성형"},
    {"item_id": 125, "title": "도예 공예단",           "desc": "도자기 공예로 식기·인테리어 소품을 제작해보는 그룹 (도자기)", "personality": "내향형, 이성형"},
    {"item_id": 126, "title": "가죽공예 스킬랩",        "desc": "가죽 재단부터 바느질까지 실습하고 디자인을 공유하는 모임 (가죽공예)", "personality": "내향형, 이성형"},
    {"item_id": 127, "title": "목공 마스터스",         "desc": "목재를 활용해 가구·장식품을 제작하며 기술을 배우는 모임 (목공예)", "personality": "내향형, 이성형"},
    {"item_id": 128, "title": "비즈 디자인 팩토리",     "desc": "비즈공예로 액세서리·장신구를 창작하는 교실 (비즈공예)", "personality": "내향형, 이성형"},
    {"item_id": 129, "title": "캔들/디퓨저 크리에이티브", "desc": "향초와 디퓨저를 예술적으로 표현해보는 창작 스튜디오 (캔들/디퓨저)", "personality": "내향형, 이성형"},
    {"item_id": 130, "title": "페이퍼아트 디자인랩",     "desc": "종이 공예로 독특한 작품과 미니어처를 만드는 연구회 (페이퍼크래프트)", "personality": "내향형, 이성형"},
    {"item_id": 131, "title": "마크라메 공예실",         "desc": "마크라메 매듭 기법을 다양하게 시도해보는 작업실 (마크라메)", "personality": "내향형, 이성형"},
    {"item_id": 132, "title": "레진아트 크리에이터즈",    "desc": "레진을 활용해 장식품·소품을 디자인하고 공유하는 모임 (레진아트)", "personality": "내향형, 이성형"}
]


In [ ]:
# (3) 아이템 임베딩 생성
###############################
def make_item_embedding_dict(items, model):
    item_embedding_dict = {}
    for item in items:
        text = f"{item['title']} {item['desc']}"
        emb = model.encode(text, convert_to_numpy=True)
        item_embedding_dict[item['item_id']] = emb
    return item_embedding_dict

item_embedding_dict = make_item_embedding_dict(items, model_bert)

In [ ]:
import re
###############################
# (4) 사용자 프로필 임베딩 생성
###############################
def make_user_profile_text(user_profile: dict) -> str:
    ext = user_profile.get("extroversion", "")
    ft  = user_profile.get("feeling_thinking", "")
    hobby = user_profile.get("hobby", "")
    detail = user_profile.get("detail_hobby", "")

    if hobby == "여행":
        return (f"저는 {ext}이며 {ft}한 성향을 가진 사람입니다. "
                f"여행을 정말 좋아하며, 특히 {detail}을 즐깁니다. "
                "다양한 문화와 맛집을 직접 체험하고 싶어합니다.")
    elif hobby == "운동":
        return (f"저는 {ext}이며 {ft}한 성향을 가진 사람입니다. "
                f"운동에 열정을 가지고 있으며, 특히 {detail}을 즐깁니다. "
                "건강과 체력을 증진시키고자 꾸준히 활동합니다.")
    elif hobby == "독서":
        return (f"저는 {ext}이며 {ft}한 성향을 가진 사람입니다. "
                f"독서를 사랑하며, 특히 {detail}에 깊은 관심을 가지고 있습니다. "
                "새로운 지식과 다양한 관점을 배우고자 합니다.")
    elif hobby == "영화":
        return (f"저는 {ext}이며 {ft}한 성향을 가진 사람입니다. "
                f"영화를 감상하는 것을 좋아하며, 특히 {detail} 장르에 큰 흥미를 느낍니다. "
                "감동과 재미를 동시에 경험하고 싶어합니다.")
    elif hobby == "게임":
        return (f"저는 {ext}이며 {ft}한 성향을 가진 사람입니다. "
                f"게임을 즐기며, 특히 {detail} 게임에 많은 관심을 가지고 있습니다. "
                "전략적 사고와 도전을 통해 성취감을 느끼고자 합니다.")
    elif hobby == "공예":
        return (f"저는 {ext}이며 {ft}한 성향을 가진 사람입니다. "
                f"공예 활동에 열정을 가지고 있으며, 특히 {detail}을 즐깁니다. "
                "창의적인 아이디어를 실현하며 섬세한 작업을 즐깁니다.")
    else:
        return f"저는 {ext}이며 {ft}한 성향을 가진 사람입니다. 다양한 취미를 즐기고 있습니다."

def get_user_embedding(user_profile: dict, model):
    user_text = make_user_profile_text(user_profile)
    user_emb = model.encode(user_text, convert_to_numpy=True)
    return user_emb

def extract_hobby(desc):
    """ 아이템 설명에서 괄호 안의 취미 정보를 안전하게 추출하는 함수 """
    match = re.search(r"\((.*?)\)", desc)
    return match.group(1) if match else ""
###############################
# (5) 코사인 유사도 및 추천
###############################
def cosine_similarity(vec1, vec2):
    """ 두 벡터의 코사인 유사도를 계산하는 함수 (예외 처리 추가) """
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    if norm1 == 0 or norm2 == 0:
        return 0  # 벡터 크기가 0이면 유사도 0 반환
    return np.dot(vec1, vec2) / (norm1 * norm2)

def recommend_content_based(user_profile, item_embedding_dict, items, top_n=5):
    user_emb = get_user_embedding(user_profile, model_bert)
    scored = []

    user_hobby = user_profile.get("hobby", "")
    user_detail_hobby = user_profile.get("detail_hobby", "")
    user_extroversion = user_profile.get("extroversion", "")
    user_feeling_thinking = user_profile.get("feeling_thinking", "")

    for item in items:
        item_id = item["item_id"]
        item_emb = item_embedding_dict[item_id]
        sim = cosine_similarity(user_emb, item_emb)

        # 아이템 정보 추출
        item_hobby = extract_hobby(item["desc"])  # 안전한 취미 추출
        item_personality = item["personality"]

        # 가중치 초기값
        weight = 1.0

        # 🎯 취미 가중치 적용
        if user_hobby and user_hobby in item_hobby:
            weight *= 1.2  # 사용자의 취미와 정확히 일치하면 1.2배 증가
        if user_detail_hobby and user_detail_hobby in item_hobby:
            weight *= 1.1  # 세부 취미 포함 시 추가 가중치 적용

        # 🎯 성향 가중치 적용
        personality_match_count = sum(trait in item_personality for trait in [user_extroversion, user_feeling_thinking])
        if personality_match_count == 1:
            weight *= 1.15  # 한 가지 성향이 일치하면 가중치 증가
        elif personality_match_count == 2:
            weight *= 1.3  # 두 가지 성향이 모두 일치하면 더 큰 가중치 적용

        # 최종 유사도 조정
        adjusted_sim = sim * weight

        scored.append((item, adjusted_sim))

    # 유사도가 높은 순으로 정렬
    scored.sort(key=lambda x: x[1], reverse=True)

    return scored[:top_n]



In [ ]:
###############################
# (6) 테스트: 사용자 프로필에 따른 추천 실행
###############################
test_user_profile = {
    "extroversion": "내향형",
        "feeling_thinking": "감정형",
        "hobby": "여행",
        "detail_hobby": "국내여행"
}

cb_recs = recommend_content_based(test_user_profile, item_embedding_dict, items, top_n=5)

print("\n=== Content-based 추천 결과 (개선 버전) ===")
for item, score in cb_recs:
    print(f"item_id={item['item_id']} | 유사도={score:.4f}")
    print(f"제목: {item['title']}")
    print(f"설명: {item['desc']}\n")



=== Content-based 추천 결과 (개선 버전) ===
item_id=85 | 유사도=1.4749
제목: 오지여행 탐험대
설명: 국내 숨겨진 오지를 찾아 떠나며 모험심을 기르는 모임 (국내여행)

item_id=92 | 유사도=1.3305
제목: 숲속 힐링 트래블
설명: 자연 속에서 치유와 재충전을 목적으로 여행하는 그룹 (힐링여행)

item_id=89 | 유사도=1.3244
제목: 골목길 투어 클럽
설명: 도시 골목길과 역사적 공간을 함께 탐방하는 여행 서클 (도시여행)

item_id=86 | 유사도=1.3060
제목: 글로벌 백팩 트립
설명: 해외 배낭여행 루트를 함께 계획하고 실행하는 동호회 (해외여행)

item_id=102 | 유사도=1.2505
제목: 아트북 인사이트회
설명: 예술 관련 서적을 통해 예술사와 작품 세계를 탐구하는 모임 (예술)

